Wrangling Cycling Raw Start and End Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
start_to_agg = pd.read_csv('df_start_flow_raw.csv')
#end_to_agg = pd.read_csv('df_end_flow_raw.csv')

In [ ]:
def to_datetime(data):
    try:
        # Parse YYYY-MM-DD HH:MM format directly
        return pd.to_datetime(data, format='%Y-%m-%d %H:%M:%S')
    except:
        # Parse DD/MM/YYYY HH:MM format
        return pd.to_datetime(data, format='%d/%m/%Y %H:%M:%S')

In [6]:
#start_to_agg['Start date'] = pd.to_datetime(start_to_agg['Start date']).dt.strftime('%Y-%m-%d')
start_to_agg['Start date'] = start_to_agg['Start date'].apply(lambda x: pd.to_datetime(x).strftime('%d/%m/%Y'))

In [10]:
start_to_agg.to_csv('df_start_flow_raw_v2.csv', index=False)

In [9]:
# Mapping of date pairs to be changed
date_changes = {
    '12/04/2022': '04/12/2022',
    '12/01/2022': '01/12/2022',
    '12/02/2022': '02/12/2022',
    '12/03/2022': '03/12/2022',
}

# Apply the changes
for original, new in date_changes.items():
    start_to_agg['Start date'] = start_to_agg['Start date'].replace(original, new)



In [11]:
start_to_agg_copy = start_to_agg.copy()

In [13]:
start_to_agg_copy['Start date'] = pd.to_datetime(start_to_agg_copy['Start date'], errors='coerce')

In [14]:
# Exclude non-numeric columns before grouping and summing
numeric_cols = start_to_agg_copy.select_dtypes(include=[np.number]).columns
cycle_start_borough_daily = start_to_agg_copy.groupby(start_to_agg_copy['Start date'].dt.date)[numeric_cols].sum()


In [16]:
cycle_start_borough_daily.to_csv('cycle_start_borough_daily_final.csv', index=True)

## Testing multi-thread processing:

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Assuming start_to_agg is your DataFrame


def convert_format_date(date_str):
    return pd.to_datetime(date_str).strftime('%d/%m/%Y')

def apply_multithreading(data, func):
    with ThreadPoolExecutor() as executor:
        # Apply the function in parallel
        result = list(executor.map(func, data))
    return result

# Apply the conversion and formatting in parallel
start_to_agg['Start date'] = apply_multithreading(start_to_agg['Start date'], convert_format_date)

In [3]:
pd.to_datetime(start_to_agg['Start date'] , format='mixed')

0         2022-04-12 23:59:00
1         2022-04-12 23:59:00
2         2022-04-12 23:58:00
3         2022-04-12 23:57:00
4         2022-04-12 23:57:00
                  ...        
8512268   2023-11-15 00:01:00
8512269   2023-11-15 00:01:00
8512270   2023-11-15 00:01:00
8512271   2023-11-15 00:00:00
8512272   2023-11-15 00:00:00
Name: Start date, Length: 8512273, dtype: datetime64[ns]

In [9]:
start_to_agg

,Start date
0,2023-11-15 00:01
1,04/12/2022 23:57


## now handle end flow:

In [1]:
import pandas as pd
import numpy as np

In [29]:
end_to_agg = pd.read_csv('df_start_flow_raw.csv')

In [ ]:
#start_to_agg['Start date'] = pd.to_datetime(start_to_agg['Start date']).dt.strftime('%Y-%m-%d')
#end_to_agg['Start date'] = end_to_agg['Start date'].apply(lambda x: pd.to_datetime(x).strftime('%d/%m/%Y'))

Date time still not correct.

In [30]:
df_slash = end_to_agg[end_to_agg['Start date'].str.contains('/')]
df_dash = end_to_agg[end_to_agg['Start date'].str.contains('-')]


In [31]:
#df_slash['Start date'] = pd.to_datetime(df_slash['Start date'], format='%d/%m/%Y/ %H:%M').dt.date
df_slash['Start date'] = pd.to_datetime(df_slash['Start date'], format='%d/%m/%Y %H:%M').dt.date



/tmp/ipykernel_370553/442898615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slash['Start date'] = pd.to_datetime(df_slash['Start date'], format='%d/%m/%Y %H:%M').dt.date


In [32]:
df_slash

,Start date,Borough Type,Borough,Barnet,Camden,City of London,Enfield,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Newham,Southwark,Tower Hamlets,Wandsworth,Westminster
0,2022-12-04,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2022-12-04,Start borough,Camden,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-12-04,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2022-12-04,Start borough,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,2022-12-04,Start borough,City of London,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84319,2022-12-01,Start borough,Camden,0,1,0,0,0,0,0,0,0,0,0,0,0,0
84320,2022-12-01,Start borough,Hackney,0,0,0,0,1,0,0,0,0,0,0,0,0,0
84321,2022-12-01,Start borough,Hammersmith and Fulham,0,0,0,0,0,1,0,0,0,0,0,0,0,0
84322,2022-12-01,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [33]:
df_slash['Start date'] = pd.to_datetime(df_slash['Start date']).dt.date

# Ensure numpy is imported
numeric_cols = df_slash.select_dtypes(include=[np.number]).columns

# Since 'Start date' is now of object type (dates), you directly group by it without using .dt
cy_end_br_daily_pt1 = df_slash.groupby('Start date')[numeric_cols].sum()

/tmp/ipykernel_370553/3180683704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slash['Start date'] = pd.to_datetime(df_slash['Start date']).dt.date


In [34]:
cy_end_br_daily_pt1

,Barnet,Camden,City of London,Enfield,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Newham,Southwark,Tower Hamlets,Wandsworth,Westminster
Start date,,,,,,,,,,,,,,
2022-12-01,50,3005,2143,160,1594,1994,949,2435,2383,43,1940,2527,1340,6727
2022-12-02,47,2397,1625,116,1551,1863,842,2150,1997,31,1618,2380,1249,5617
2022-12-03,52,1485,943,82,1488,1550,593,1877,1615,22,1323,2189,1196,4668
2022-12-04,34,1106,738,65,1119,1217,448,1311,1205,15,1048,1738,939,3485


In [35]:
#
df_dash['Start date'] = pd.to_datetime(df_dash['Start date'], format='%Y-%m-%d %H:%M').dt.date


/tmp/ipykernel_370553/597988539.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dash['Start date'] = pd.to_datetime(df_dash['Start date'], format='%Y-%m-%d %H:%M').dt.date


In [38]:
df_dash

,Start date,Borough Type,Borough,Barnet,Camden,City of London,Enfield,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Newham,Southwark,Tower Hamlets,Wandsworth,Westminster
84324,2022-12-11,Start borough,Hammersmith and Fulham,0,0,0,0,0,1,0,0,0,0,0,0,0,0
84325,2022-12-11,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
84326,2022-12-11,Start borough,City of London,0,0,1,0,0,0,0,0,0,0,0,0,0,0
84327,2022-12-11,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
84328,2022-12-11,Start borough,Tower Hamlets,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8512268,2023-11-15,Start borough,Hammersmith and Fulham,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8512269,2023-11-15,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8512270,2023-11-15,Start borough,Westminster,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8512271,2023-11-15,Start borough,Camden,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
df_dash['Start date'] = pd.to_datetime(df_dash['Start date']).dt.date

# Ensure numpy is imported
numeric_cols_2 = df_dash.select_dtypes(include=[np.number]).columns

# Since 'Start date' is now of object type (dates), you directly group by it without using .dt
cy_end_br_daily_pt2 = df_dash.groupby('Start date')[numeric_cols].sum()

/tmp/ipykernel_370553/1687351496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dash['Start date'] = pd.to_datetime(df_dash['Start date']).dt.date


In [39]:
cy_end_br_daily_pt2

,Barnet,Camden,City of London,Enfield,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Newham,Southwark,Tower Hamlets,Wandsworth,Westminster
Start date,,,,,,,,,,,,,,
2022-12-05,48,2592,1919,135,1349,1819,795,2035,1905,24,1502,2146,1189,5847
2022-12-06,71,2852,2177,144,1507,1854,885,2236,2223,38,1682,2401,1356,6373
2022-12-07,53,2773,2012,150,1476,1942,891,2048,2091,32,1582,2326,1192,5954
2022-12-08,53,2564,1822,135,1404,1803,749,1950,2035,28,1534,2047,1111,5380
2022-12-09,49,2200,1501,126,1429,1664,726,1936,1754,26,1493,2166,1132,5234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-26,21,891,644,60,952,991,433,1142,944,18,944,1594,735,2946
2023-11-27,41,1921,1479,100,1069,1511,750,1696,1698,19,1327,1743,1013,4481
2023-11-28,57,2805,2114,146,1465,1955,995,2218,2370,29,1755,2385,1278,6154


In [40]:
merged_df_end = pd.concat([cy_end_br_daily_pt2, cy_end_br_daily_pt1])


In [28]:
#merged_df_end.to_csv('cycle_end_borough_daily_final.csv', index=True)

In [41]:
merged_df_end.to_csv('cycle_start_borough_daily_final_v2.csv', index=True)

In [3]:
from concurrent.futures import ThreadPoolExecutor


# Assuming start_to_agg is your DataFrame
start_to_agg = pd.DataFrame({'Start date': ['2023-11-15 00:01', '04/12/2022 23:57']})

def convert_format_date(date_str):
    return pd.to_datetime(date_str).strftime('%d/%m/%Y')

def apply_multithreading(data, func):
    with ThreadPoolExecutor() as executor:
        # Apply the function in parallel
        result = list(executor.map(func, data))
    return result

# Apply the conversion and formatting in parallel
end_to_agg['Start date'] = apply_multithreading(end_to_agg['Start date'], convert_format_date)

In [4]:
date_changes = {
    '12/04/2022': '04/12/2022',
    '12/01/2022': '01/12/2022',
    '12/02/2022': '02/12/2022',
    '12/03/2022': '03/12/2022',
}

# Apply the changes
for original, new in date_changes.items():
    end_to_agg['Start date'] = end_to_agg['Start date'].replace(original, new)

In [5]:
end_to_agg_copy = end_to_agg.copy()

In [6]:
end_to_agg_copy['Start date'] = pd.to_datetime(end_to_agg_copy['Start date'], errors='coerce')

In [7]:
# Exclude non-numeric columns before grouping and summing
numeric_cols = end_to_agg_copy.select_dtypes(include=[np.number]).columns
cycle_start_borough_daily = end_to_agg_copy.groupby(end_to_agg_copy['Start date'].dt.date)[numeric_cols].sum()


In [8]:
cycle_start_borough_daily

,Barnet,Camden,City of London,Enfield,Hackney,Hammersmith and Fulham,Islington,Kensington and Chelsea,Lambeth,Newham,Southwark,Tower Hamlets,Wandsworth,Westminster
Start date,,,,,,,,,,,,,,
2022-01-12,50,3005,2143,160,1594,1994,949,2435,2383,43,1940,2527,1340,6727
2022-02-12,47,2397,1625,116,1551,1863,842,2150,1997,31,1618,2380,1249,5617
2022-03-12,52,1485,943,82,1488,1550,593,1877,1615,22,1323,2189,1196,4668
2022-04-12,34,1106,738,65,1119,1217,448,1311,1205,15,1048,1738,939,3485
2022-05-12,48,2592,1919,135,1349,1819,795,2035,1905,24,1502,2146,1189,5847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-07,71,2833,2423,190,1965,2146,1197,2683,2924,49,2251,3225,1674,8289
2023-12-08,43,1447,1035,126,1658,1634,691,1869,1570,38,1584,2652,1406,5816
2023-12-09,54,2804,2548,153,1784,2341,1054,2517,2781,41,2092,2941,1655,7260
